In [1]:
from rdflib import ConjunctiveGraph
from jinja2 import Template
import json
from pyshacl import validate
from IPython.display import display, Markdown, Latex

# 1. Defining a basic profile
Here we state that for ContactPoint, we must have a name and an email. We also state that a Dataset must have a name a description a datePublished and a license. 

In [24]:
sample_profile = {'ContactPoint': ['name', 'email'], 
                 'Dataset': ['name', 'description', 'datePublished', 'license']}


# 2. Generating a SHACL shape from a this predefined profile

In [32]:
shape_template = """
@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix edam: <http://edamontology.org/> .
@prefix biotools: <https://bio.tools/ontology/> .

{# comment: here we generate one shape per class in the profile #}
{% for class_name in profile.keys() %}
schema:{{class_name}}Shape
    a sh:NodeShape ;
    sh:targetClass schema:{{class_name}} ;

    {# comment: iterate over mandatory properties #}
    {% for prop_name in profile[class_name] %}
    sh:property [
        sh:path schema:{{prop_name}} ;
        sh:minCount 1 ;
    ] ;
    {% endfor %}
    .
{% endfor %}
"""

template = Template(shape_template)
shape = template.render(profile=sample_profile)
print(shape)
g = ConjunctiveGraph()
g.parse(data = shape, format='turtle')
print(len(g))


@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix edam: <http://edamontology.org/> .
@prefix biotools: <https://bio.tools/ontology/> .



schema:ContactPointShape
    a sh:NodeShape ;
    sh:targetClass schema:ContactPoint ;

    
    
    sh:property [
        sh:path schema:name ;
        sh:minCount 1 ;
    ] ;
    
    sh:property [
        sh:path schema:email ;
        sh:minCount 1 ;
    ] ;
    
    .

schema:DatasetShape
    a sh:NodeShape ;
    sh:targetClass schema:Dataset ;

    
    
    sh:property [
        sh:path schema:name ;
        sh:minCount 1 ;
    ] ;
    
    sh:property [
        sh:path schema:description ;
        sh:minCount 1 ;
    ] ;
    
    sh:property [
        sh:path schema:datePublished ;
        

# 3. Applying the SHACL rule to validate a sample RO-crate

In [34]:
ro_path = 'sample_data/clinvap/ro-crate-metadata.jsonld'
#ro = ConjunctiveGraph()
#ro.parse('sample_data/clinvap/ro-crate-metadata.jsonld', format="json-ld")
#print(ro.serialize(format="turtle").decode())

r = validate(data_graph = ro_path, 
             data_graph_format='json-ld', 
             shacl_graph = shape, 
             shacl_graph_format = 'turtle', 
             ont_graph = None, 
             inference = 'rdfs', 
             abort_on_error = False, 
             meta_shacl = False, 
             debug = True)

conforms, results_graph, results_text = r

#print(results_text)

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema:name ]
	Focus Node: <file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/assets/>
	Result Path: schema:name

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema:name ]
	Focus Node: <file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/bin/>
	Result Path: schema:name

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema:name ]
	Focus Node: <file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/

# 4. Producing an explaination for the validation graph

In [37]:
report_query = """
    SELECT ?node ?path WHERE {
        ?v rdf:type sh:ValidationReport ;
           sh:result ?r .
        ?r sh:focusNode ?node ;
           sh:sourceShape ?s . 
        ?s sh:path ?path . 
    }
"""

results = results_graph.query(report_query)

print(str(len(results))+ ' actions needed to fix RO-crate mandatory fields')

for r in results :
    display(Markdown('The RO-crate `{}` should be fixed, it is missing information for field {}'.format(str(r['node']), str(r['path']))))
    


18 actions needed to fix RO-crate mandatory fields


The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/docs/` should be fixed, it is missing information for field http://schema.org/name

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/bin/` should be fixed, it is missing information for field http://schema.org/license

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/` should be fixed, it is missing information for field http://schema.org/datePublished

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/` should be fixed, it is missing information for field http://schema.org/license

...

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/bin/` should be fixed, it is missing information for field http://schema.org/datePublished

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/assets/` should be fixed, it is missing information for field http://schema.org/datePublished

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/conf/` should be fixed, it is missing information for field http://schema.org/description

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/docs/` should be fixed, it is missing information for field http://schema.org/datePublished

The RO-crate `file:///Users/gaignard-a/Documents/Dev/ro-crate-py/notebooks/sample_data/clinvap/conf/` should be fixed, it is missing information for field http://schema.org/license